In [ ]:
### Importing libraries ###

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.distributions.bernoulli import Bernoulli

import torchvision
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST
from torchvision import transforms

import matplotlib.pyplot as plt
import time
import pickle as pkl

import import_ipynb
from TNN import *

In [ ]:
### A single TNN column with EC, LI and STDP ###

class TNNColumn():
    def __init__(self, tin_max, p, q, wres, theta, ntype, stdp, stdp_random, w_init):
        self.time       = tin_max
        self.wmax       = 2**(wres)-1
        self.p          = p
        self.q          = q
        self.wres       = wres
        self.theta      = theta
        self.ntype      = ntype
        self.w_init     = w_init
        self.ec         = ExcitatoryColumn(self.time, self.p, self.q, self.wres, self.theta, self.ntype, self.w_init)
        if stdp_random:
            if stdp == "stdp":
                self.stdp   = STDP(wres)
            elif stdp == "rstdp":
                self.stdp   = RSTDP(wres)
        else:
            if stdp == "stdp":
                self.stdp   = STDP_Det(wres)
            elif stdp == "rstdp":
                self.stdp   = RSTDP_Det(wres)
            elif stdp == "stdp_tmod":
                self.stdp   = STDP_Det_TMod(wres, self.time + self.wmax)
            elif stdp == "stdp_smod":
                self.stdp   = STDP_Det_SMod(wres, self.time + self.wmax)
            elif stdp == "rstdp_tmod":
                self.stdp   = RSTDP_Det_TMod(wres, self.time + self.wmax)    
            elif stdp == "stdp_recc":
                self.stdp   = STDP_Det_Recc(wres, self.time + self.wmax)
        self.inhib      = LateralInhibition()
    def __call__(self, data, k):
        data            = torch.flatten(data)
        ec_out          = self.ec(data)
        li_out, winidx  = self.inhib(ec_out, k)
        return li_out, winidx